In [14]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import Neumann
from torch.utils.data import DataLoader,sampler,Dataset
from torch.autograd import Variable

In [2]:
batch_size = 48
test_batch_size = 1000
seed = 123
momentum = 0.5
log_interval = 10

torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

In [3]:
class SimpleDataset(Dataset):
    """Action dataset."""

    def __init__(self, data_file):
        """
        Args:
            root_dir (string): Directory with all the data is stored.
            labels(list): GT
        """
        self.data_file = data_file
        self.data = np.loadtxt(self.data_file, delimiter=',')#, dtype={'names': ('X', 'Y'), 'formats': ('S1', 'S1')})
        self.length = self.data.shape[0]
        self.labels = self.data[:, -1]
        self.data = self.data[:,0]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        sample={'X':self.data[idx], 'Y' : self.labels[idx]}
#         print(sample)
        return sample
    

In [4]:
simple_dataset = SimpleDataset("./slr10.csv")

train_loader = torch.utils.data.DataLoader(
    simple_dataset,
    batch_size=batch_size, shuffle=True)

# test_loader = torch.utils.data.DataLoader(
#     batch_size=test_batch_size, shuffle=True, **kwargs)

In [7]:
class LinearRegressionModel(nn.Module):

    def __init__(self, input_dim, output_dim):

        super(LinearRegressionModel, self).__init__() 
        # Calling Super Class's constructor
        self.linear = nn.Linear(input_dim, output_dim)
        # nn.linear is defined in nn.Module

    def forward(self, x):
        # Here the forward pass is simply a linear function

        out = self.linear(x)
        return out

input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim,output_dim)

criterion = nn.MSELoss()# Mean Squared Loss
l_rate = 0.001

optimiser = Neumann(list(model.parameters()), lr = l_rate) #Stochastic Gradient Descent
    

In [10]:
epochs = 100

for epoch in range(epochs):
    for i, sample in enumerate(train_loader):

        #increase the number of epochs by 1 every time
        inputs = Variable(sample['X'].type(torch.FloatTensor))
        labels = Variable(sample['Y'].type(torch.FloatTensor))

        #clear grads as discussed in prev post
        optimiser.zero_grad()
        #forward to get predicted values
        outputs = model.forward(inputs.view(-1, 1))
#         loss = criterion(outputs, labels)
#         print(outputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()# back props
        optimiser.step()# update the parameters
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))

epoch 0, loss 17.191287994384766
epoch 1, loss 16.576515197753906
epoch 2, loss 15.96240234375
epoch 3, loss 15.351107597351074
epoch 4, loss 14.744500160217285
epoch 5, loss 14.144200325012207
epoch 6, loss 13.551612854003906
epoch 7, loss 12.967961311340332
epoch 8, loss 12.394291877746582
epoch 9, loss 11.831509590148926
epoch 10, loss 11.280391693115234
epoch 11, loss 10.741596221923828
epoch 12, loss 10.215685844421387
epoch 13, loss 9.703120231628418
epoch 14, loss 9.204290390014648
epoch 15, loss 8.719508171081543
epoch 16, loss 8.249022483825684
epoch 17, loss 7.793018341064453
epoch 18, loss 7.351638317108154
epoch 19, loss 6.924966335296631
epoch 20, loss 6.899356842041016
epoch 21, loss 6.866124629974365
epoch 22, loss 6.825706481933594
epoch 23, loss 6.778513431549072
epoch 24, loss 6.724961757659912
epoch 25, loss 6.665435314178467
epoch 26, loss 6.600317001342773
epoch 27, loss 6.5299763679504395
epoch 28, loss 6.454763889312744
epoch 29, loss 6.375024318695068
epoch 30, 

In [17]:
 for i, sample in enumerate(train_loader):

        #increase the number of epochs by 1 every time
    print(sample)

{'X': tensor([ 3.4000,  3.7000,  3.4000,  3.2000], dtype=torch.float64), 'Y': tensor([ 4.6000,  5.1000,  5.2000,  4.7000], dtype=torch.float64)}
{'X': tensor([ 4.4000,  3.5000,  3.2000,  3.8000], dtype=torch.float64), 'Y': tensor([ 5.7000,  5.5000,  4.7000,  5.1000], dtype=torch.float64)}
{'X': tensor([ 3.0000,  3.1000,  3.8000,  3.4000], dtype=torch.float64), 'Y': tensor([ 4.8000,  4.6000,  4.6000,  5.0000], dtype=torch.float64)}
{'X': tensor([ 3.8000,  3.4000,  4.0000,  3.1000], dtype=torch.float64), 'Y': tensor([ 5.7000,  5.0000,  5.8000,  4.9000], dtype=torch.float64)}
{'X': tensor([ 3.5000,  3.5000,  3.5000,  3.2000], dtype=torch.float64), 'Y': tensor([ 5.0000,  5.1000,  5.2000,  5.0000], dtype=torch.float64)}
{'X': tensor([ 3.5000,  3.0000,  3.9000,  3.4000], dtype=torch.float64), 'Y': tensor([ 5.1000,  4.9000,  5.4000,  5.4000], dtype=torch.float64)}
{'X': tensor([ 3.8000,  3.9000,  3.4000,  3.3000], dtype=torch.float64), 'Y': tensor([ 5.1000,  5.4000,  5.1000,  5.1000], dtype=t

In [12]:
predicted =model.forward(Variable(torch.from_numpy(X_train))).data.numpy()

plt.plot(x_train, y_correct, 'go', label = 'from data', alpha = .5)
plt.plot(x_train, predicted, label = 'prediction', alpha = 0.5)
plt.legend()
plt.show()
print(model.state_dict())

NameError: name 'X_train' is not defined